In [153]:
import pandas as pd

In [154]:
df=pd.read_csv('C:/Users/pedro/Projetos/bundesbet/src/data/game_stats.csv')

In [155]:
df.head(1)

,url,round,home_team,away_team,goals_home_final,goals_away_final,final_result,goals_home_1half,goals_away_1half,1half_result,...,tackleshome,tacklesaway,attackshome,attacksaway,dangerous_attackshome,dangerous_attacksaway,red_cardshome,red_cardsaway,completed_passeshome,completed_passesaway
0,https://www.scoreboard.com/uk/match/OjVQbVCq/#...,26,Frankfurt,Monchengladbach,1,3,Away,0,2,Away,...,4.0,4.0,84,35,21,6,NaN,NaN,NaN,NaN


In [185]:
def get_wins(team,rnd,df):
    mask1 = (df['home_team']== team) & (df['final_result'] == 'Home') & (df['round']<= rnd)
    mask2 = (df['away_team']== team) & (df['final_result'] == 'Away') & (df['round']<= rnd)
    return len(df.loc[mask1|mask2,'round'])
def get_games(team,rnd,df):
    mask1 = ((df['home_team']== team) | (df['away_team']== team)) & (df['round']<= rnd)
    return len(df.loc[mask1,'round'])
def get_draws(team,rnd,df):
    mask1 = ((df['home_team']== team) | (df['away_team']== team)) & (df['final_result'] == 'Draw') & (df['round']<= rnd)
    return len(df.loc[mask1,'round'])
def get_loses(team,rn,dfd):
    mask1 = (df['home_team']== team) & (df['final_result'] == 'Away') & (df['round']<= rnd)
    mask2 = (df['away_team']== team) & (df['final_result'] == 'Home') & (df['round']<= rnd)
    return len(df.loc[mask1|mask2,'round'])
def gp(team,rnd,df):
    goals=df.query(f'home_team == "{team}" & round <= {rnd}')['goals_home_final'].sum() + df.query(f'away_team == "{team}" & round <= {rnd}')['goals_away_final'].sum()
    return goals
def gc(team,rnd,df):
    goals=df.query(f'away_team == "{team}" & round <= {rnd}')['goals_home_final'].sum() + df.query(f'home_team == "{team}" & round <= {rnd}')['goals_away_final'].sum()
    return goals
def get_points(row):
    points= row['wins']*3+row['draws']
    return points

In [184]:
def ger_table(rnd,df):
    data={}
    data['teams']=list(df['home_team'].unique())
    data['games']=[get_games(team, rnd, df) for team in data['teams']]
    data['wins']=[get_wins(team, rnd, df) for team in data['teams']]
    data['draws']=[get_draws(team, rnd, df) for team in data['teams']]
    data['loses']=[get_loses(team, rnd, df) for team in data['teams']]
    data['gp']=[gp(team, rnd, df) for team in data['teams']]
    data['gc']=[gc(team, rnd, df) for team in data['teams']]
    table=pd.DataFrame(data)
    table['sg']=table['gp']-table['gc']
    table['points']=table.apply(lambda row: get_points(row), axis=1)
    table=table.sort_values(by=['points'], ascending=False).reset_index(drop=True)
    table.index = np.arange(1, len(table)+1)
    return table

In [183]:
table = ger_table(2)
table

,teams,games,wins,draws,loses,gp,gc,sg,points
1,Wolfsburg,2,2,0,0,5,1,4,6
2,Dortmund,2,2,0,0,8,2,6,6
3,Freiburg,2,2,0,0,6,1,5,6
4,RB Leipzig,2,2,0,0,6,1,5,6
5,Leverkusen,2,2,0,0,6,3,3,6
6,Bayern,2,1,1,0,5,2,3,4
7,Monchengladbach,2,1,1,0,3,1,2,4
8,Frankfurt,2,1,0,1,2,2,0,3
9,Dusseldorf,2,1,0,1,4,4,0,3
10,Hoffenheim,2,1,0,1,3,3,0,3
